# Lab: Cross-Validation and the Bootstrap

## Load Libraries

In [1]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize , poly)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import (cross_validate, KFold, ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

## Load data

In [3]:
Auto = load_data('Auto')

Auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


## Separate train and test data

In [4]:
Auto_train, Auto_valid = train_test_split(
                                            Auto,
                                            test_size = 196,
                                            random_state = 0
)

### Model first model

In [5]:
hp_mm = MS(['horsepower'])

In [6]:
XTrain = hp_mm.fit_transform(Auto_train)

In [7]:
yTrain = Auto_train['mpg']

In [9]:
model = sm.OLS(yTrain, XTrain)

In [10]:
results = model.fit()

### Check results

In [12]:
XValid = hp_mm.fit_transform(Auto_valid)

In [13]:
yValid = Auto_valid['mpg']

In [15]:
valid_pred = results.predict(XValid)

In [17]:
np.mean((yValid - valid_pred)**2)

23.61661706966988

### Function to calculate RMSE

In [18]:
def evalMSE(terms, response, train, test):
    
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    X_test = mm.transform(test)
    y_test = test[response]
    results = sm.OLS(y_train , X_train).fit()
    test_pred = results.predict(X_test)
    
    return np.mean((y_test - test_pred)**2)

## Test different orders of polynomials

In [26]:
MSE = np.zeros(3)

In [28]:
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE(
                        [poly('horsepower', degree)],
                        'mpg',
                        Auto_train,
                        Auto_valid
    )

In [29]:
MSE

array([23.61661707, 18.76303135, 18.79694163])

## Cross-validation

In [32]:
hp_model = sklearn_sm(
                        sm.OLS,
                        MS(['horsepower'])
)

X, Y = Auto.drop(columns=['mpg']), Auto['mpg']

cv_results = cross_validate(
                                hp_model,
                                X,
                                Y,
                                cv = Auto.shape[0] # LOOCV
)
                                                
cv_err = np.mean(cv_results['test_score'])

cv_err

24.23151351792922

In [33]:
cv_error = np.zeros(5)

In [36]:
H = np.array(Auto['horsepower'])

In [37]:
M = sklearn_sm(sm.OLS)

In [41]:
for i, d in enumerate(range(1, 6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(
                            M,
                            X,
                            Y,
                            cv = Auto.shape[0] # LOOCV
    )
    cv_error[i] = np.mean(M_CV['test_score'])

In [42]:
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.4244303 , 19.03322411])

In [43]:
cv_error = np.zeros(5)

In [46]:
# This code only configures the CV

cv = KFold(
            n_splits = 10,
            shuffle = True,
            random_state = 0
)

In [47]:
for i, d in enumerate(range(1, 6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(
                            M,
                            X,
                            Y,
                            cv = cv # 10 splits
    )
    cv_error[i] = np.mean(M_CV['test_score'])

In [48]:
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848402, 19.13719154])

In [49]:
validation = ShuffleSplit(
                        n_splits=1,
                        test_size=196,
                        random_state=0
)

In [50]:
results = cross_validate(
                        hp_model ,
                        Auto.drop(['mpg'], axis=1),
                        Auto['mpg'],
                        cv=validation
)

In [51]:
results['test_score']

array([23.61661707])

In [52]:
validation = ShuffleSplit(
                        n_splits=10,
                        test_size=196,
                        random_state=0
)

In [53]:
results = cross_validate(
                        hp_model ,
                        Auto.drop(['mpg'], axis=1),
                        Auto['mpg'],
                        cv=validation
)

In [54]:
results['test_score'].mean(), results['test_score'].std()

(23.802232661034164, 1.4218450941091831)

## Bootstrap

In [56]:
Portfolio = load_data('Portfolio')

In [57]:
def alpha_func(D, idx):
    cov_ = np.cov(D[['X', 'Y']].loc[idx], rowvar = False)
    return((cov_[1, 1] - cov_[0, 1]) / (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

In [59]:
alpha_func(Portfolio, range(100))

0.57583207459283

In [62]:
# Selecting with replacement

rng = np.random.default_rng(0) # Creates a generator

alpha_func(
            Portfolio,
            rng.choice(100, 100, replace = True)
)


0.6074452469619004

In [68]:
# Standard error for the bootstrap

def boot_SE(func, D, n = None, B = 1000, seed = 0):
    rng = np.random.default_rng(seed) # Creates a generator
    first_, second_ = 0, 0 # Initialize the values first and second
    n = n or D.shape[0] # Uses shape in case of n = None
    for _ in range(B):
        idx = rng.choice(D.index, n, replace = True) # Choice 
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)    


In [69]:
alpha_SE = boot_SE(alpha_func, Portfolio, B = 1000, seed = 0)

In [70]:
alpha_SE

0.09118176521277699

In [87]:
def boot_OLS(model_matrix , response , D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

In [90]:
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')

In [91]:
rng = np.random.default_rng(0)
np.array([hp_func(Auto, rng.choice(392, 392, replace=True)) for _ in range(10)])

array([[39.88064456, -0.1567849 ],
       [38.73298691, -0.14699495],
       [38.31734657, -0.14442683],
       [39.91446826, -0.15782234],
       [39.43349349, -0.15072702],
       [40.36629857, -0.15912217],
       [39.62334517, -0.15449117],
       [39.0580588 , -0.14952908],
       [38.66688437, -0.14521037],
       [39.64280792, -0.15555698]])

In [92]:
hp_se = boot_SE(hp_func, Auto, B = 1000, seed = 10)

In [93]:
hp_se

intercept     0.848807
horsepower    0.007352
dtype: float64